In [0]:
def clean_string(string1):
  char1 = "["
  char2 = "]"
  mystr = string1
  return mystr[mystr.find(char1)+1 : mystr.find(char2)]

def clean_commas(string1):
  string1 = string1[1:]
  string1 = string1[:-1]
  return string1

def clean(string1):
  string1 = clean_string(string1)
  string1 = clean_commas(string1)
  return string1

def clean_price(string1):
  string1 = clean_string(string1)
  string1 = clean_commas(string1)
  string1 = string1.split(' ')[1]
  string1 = string1.replace(',', '.', 1)
  return string1

def comma_dot(string1):
  string1 = string1.replace(',', '.', 1)
  return string1

def delete_n(string1):
  string1 = string1.replace('\n', '')
  return string1

In [0]:
#ci permette di importare qualsiasi modulo
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/TensorFlow 2.0/modules')
import pandas as pd
import tf_dataset_extractor as e
#import grapher_v1_1 as g
#import LSTM_creator_v1_0 as l
v = e.v
g = e.g
l = e.l

In [66]:
import pandas as pd
X = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/20200612_Wine_Price_Estimator/wines_.csv')
X = X.drop(['Unnamed: 0', '8', '9'], axis=1)

X['0'] = X['0'].apply(lambda x : clean(x))
X['1'] = X['1'].apply(lambda x : clean(x))
X['2'] = X['2'].apply(lambda x : clean(x))
X['3'] = X['3'].apply(lambda x : clean(x))
X['3'] = X['3'].apply(lambda x : delete_n(x))
X['4'] = X['4'].apply(lambda x : clean(x))
X['5'] = X['5'].apply(lambda x : clean(x))
X['6'] = X['6'].apply(lambda x : clean(x))
X['7'] = X['7'].apply(lambda x : clean(x))
X['Unnamed: 11'] = X['Unnamed: 11'].apply(lambda x : clean_price(x))
X['Unnamed: 11'] = X['Unnamed: 11'].apply(lambda x : float(x))
#X.loc[math.isnan(X['8']) == True, '8'] = 10
X
#if oginal price is None, take the current price
#X = X.dropna(axis=0)
X.columns = ['Producer', 'Kind', 'Year', 'Location', 'Region', 'Percentage', 'Alchohol', 'Volume', 'Price']
X.pop('Percentage')
#drop N/A column
X = X[~X['Year'].isin(['N/A'])]
#count unique values in Location

X['Alchohol'] = X['Alchohol'].apply(lambda x : x[0:4])
X['Alchohol'] = X['Alchohol'].apply(lambda x : float(comma_dot(x)))

e.X = X
v.partition.one_hot(['Location', 'Producer', 'Kind', 'Year', 'Region','Volume'])
e.X

,Alchohol,Price,Location_ \nDOCG Valtellina Superiore inferno\n,Location_ \nIGT Rubicone\n,Location_\n DO Bierzo\n,Location_\n DOC Primitivo di Manduria\n,Location_\n DOC Romagna Sangiovese Superiore\n,Location_\n DOCG Chianti Classico\n,Location_\n DOCG Chianti\n,Location_\n DOCG Vino Nobile di Montepulciano\n,Location_\n IGT Puglia\n,Location_\n IGT Terre Siciliane\n,Location_\n IGT Toscano o Toscana\n,"Location_\n', '\n",Location_\nAOC Bordeaux Superieur Rouge\n,Location_\nAOC Bourgogne Epineuil\n,Location_\nAOC Bourgogne Rouge\n,Location_\nAOC Bourgogne Tonnerre\n,Location_\nAOC Bourgueil\n,Location_\nAOC Brouilly\n,Location_\nAOC Chassagne-Montrachet\n,Location_\nAOC Chinon\n,Location_\nAOC Coteaux du Loir\n,Location_\nAOC Cotes de Provence Rouge\n,Location_\nAOC Crozes-Hermitage\n,Location_\nAOC Listrac-Médoc\n,Location_\nAOC Madiran Rouge\n,Location_\nAOC Morgon\n,Location_\nAOC Pessac Léognan\n,Location_\nAOC Rully Rouge\n,Location_\nAOC Saint Joseph\n,Location_\nAOC Vin de Savoie Arbin\n,Location_\nDO Bierzo\n,Location_\nDO Maipo\n,Location_\nDO Rias Baixas\n,Location_\nDO Ribeira Sacra\n,Location_\nDO Ribera del Duero\n,Location_\nDOC Aglianico del Vulture\n,Location_\nDOC Alghero Cabernet Riserva\n,Location_\nDOC Alto Adige Lagrein Riserva\n,...,Region_Bourgogne,Region_Burgenland,Region_Calabria,Region_California,Region_Campania,Region_Casablanca Valley,Region_Castilla and Leon,Region_Catalunya,Region_Colchagua Valley,Region_Emilia-Romagna,Region_Friuli-Venezia Giulia,Region_Galicia,Region_Languedoc-Roussillon,Region_Lazio,Region_Liguria,Region_Loire,Region_Lombardy,Region_Maipo Valley,Region_Marche,Region_Mendoza,Region_Molise,Region_Oregon,Region_Patagonia,Region_Piedmont,Region_Provence,Region_Rapel Valley,Region_Rhône,Region_Rioja,Region_Sardinia,Region_Savoie,Region_Sicily,Region_South Africa,Region_South Australia,Region_Southwest,Region_Trentino Alto Adige,Region_Tuscany,Region_Umbria,Region_Veneto,"Volume_0,75 l Standard","Volume_1,50 l Magnum"
1,13.0,11.31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,13.5,10.91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,13.0,11.96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,13.5,16.47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
5,13.0,9.04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,14.5,34.16,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
971,12.5,14.19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
972,13.5,17.36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
973,12.5,8.59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [67]:
y = X.pop('Price')
y = pd.DataFrame(y)
y

,Price
1,11.31
2,10.91
3,11.96
4,16.47
5,9.04
...,...
970,34.16
971,14.19
972,17.36
973,8.59


In [0]:
#splitting
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(e.X, y, test_size=0.1, random_state=42)

In [78]:
y_test

,Price
897,59.90
475,19.36
171,12.07
412,24.58
73,12.63
...,...
613,16.02
827,15.34
291,21.07
513,26.39


In [81]:
#SVR, manual procedure
from sklearn.svm import SVR
clf = SVR(kernel='poly', C=1, gamma='auto', degree=3, epsilon=.1, coef0=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test) #???
from sklearn.metrics import r2_score
#r2 score
print(r2_score(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9999898261947847


In [85]:
#performance evalutation
v.cross_validation(clf, e.X, y, 10)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

('Accuracy: 1.00 (+/- 0.00)',
 array([0.99993092, 0.9999765 , 0.99997714, 0.99997937, 0.99999018,
        0.99998762, 0.99998679, 0.99992609, 0.99997442, 0.99831325]))

In [86]:
#comparing results
y_test = y_test.reset_index()
y_test.pop('index')
y_test

y_pred = pd.DataFrame(y_pred)
total = pd.concat([y_test, y_pred], axis=1)
total

,Price,0
0,59.90,59.918099
1,19.36,19.227471
2,12.07,12.141629
3,24.58,24.603315
4,12.63,12.662931
...,...,...
92,16.02,15.995917
93,15.34,15.355782
94,21.07,21.080608
95,26.39,26.357549
